In [2]:
import pandas as pd
import numpy as np

In [19]:
def player_contracts(filepath):
    df = pd.read_csv(filepath)
    df = df.fillna(0.0)
    for col in df.iloc[:, 3:10].columns:
        df[col] = df[col].replace('[\$,]', '', regex=True).astype(int)

    current_season = df.iloc[:, 3:10].columns[0]
    next_season = df.iloc[:, 3:10].columns[1]

    indicator_col = f"{current_season}_contract_year"

    # Contract year indicator: 1 if salary > 0 this year and salary = 0 next year
    df[indicator_col] = (df[current_season].astype(float) > 0) & (df[next_season].astype(float) == 0)

    df[indicator_col] = df[indicator_col].astype(int)
    return df

salaries23 = player_contracts('nba_player_contracts/nba_contracts_2023.csv')
salaries24 = player_contracts('nba_player_contracts/nba_contracts_2024.csv')
salaries25 = player_contracts('nba_player_contracts/nba_contracts_2025.csv')


<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
/var/folders/kk/xndsxk6n7js53cqtn6q1fjkm0000gn/T/ipykernel_865/287330119.py:5: SyntaxWarning: invalid escape sequence '\$'
  df[col] = df[col].replace('[\$,]', '', regex=True).astype(int)


In [24]:
# Create 'Years of Experience' Feature
seasons = list(range(2002, 2025))
player_first_year = {}

for year in seasons:
    df = pd.read_csv(f"nba_merged_stats/nba_merged_{year}.csv")
    players = df['Player'].dropna().unique()
    for player in players:
        if player not in player_first_year:
            player_first_year[player] = year

target_year = 2023
df_2023 = pd.read_csv(f"nba_merged_stats/nba_merged_2023.csv")
df_2023['Experience'] = df_2023['Player'].map(lambda x: (2023 + 1) - player_first_year.get(x, 2023))

target_year = 2024
df_2024 = pd.read_csv(f"nba_merged_stats/nba_merged_2024.csv")
df_2024['Experience'] = df_2024['Player'].map(lambda x: (2024 + 1) - player_first_year.get(x, 2024))

In [25]:
# Indicators from Awards
def check_award_winner(x):
    awards = ["MVP-1", "DPOY-1", "MIP-1", "CPOY-1", "ROY-1", "6MOY-1"]
    for a in awards:
        if a in x.split(','):
            return 1
    return 0

def all_nba_team_1(x):
    award = 'NBA1'
    if award in x.split(','):
        return 1
    return 0

def all_nba_team_2(x):
    award = 'NBA2'
    if award in x.split(','):
        return 1
    return 0

def all_nba_team_3(x):
    award = 'NBA3'
    if award in x.split(','):
        return 1
    return 0

def all_defensive_1(x):
    award = 'DEF1'
    if award in x.split(','):
        return 1
    return 0

def all_defensive_2(x):
    award = 'DEF2'
    if award in x.split(','):
        return 1
    return 0


In [27]:

df_2024['Season'] = 2024
df_2024['NumOfAwards'] = df_2024['Awards'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)
df_2024["All-Star"] = df_2024['Awards'].apply(lambda x: ("AS" in x) + 0 if pd.notna(x) else 0)
df_2024['AwardWinner'] = df_2024['Awards'].apply(lambda x: check_award_winner(x) if pd.notna(x) else 0)
df_2024['FirstTeam'] = df_2024['Awards'].apply(lambda x: all_nba_team_1(x) if pd.notna(x) else 0)
df_2024['SecondTeam'] = df_2024['Awards'].apply(lambda x: all_nba_team_2(x) if pd.notna(x) else 0)
df_2024['ThridTeam'] = df_2024['Awards'].apply(lambda x: all_nba_team_3(x) if pd.notna(x) else 0)
df_2024['DefTeam1'] = df_2024['Awards'].apply(lambda x: all_defensive_1(x) if pd.notna(x) else 0)
df_2024['DefTeam2'] = df_2024['Awards'].apply(lambda x: all_defensive_2(x) if pd.notna(x) else 0)

In [28]:

df_2023['Season'] = 2023
df_2023['NumOfAwards'] = df_2023['Awards'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)
df_2023["All-Star"] = df_2023['Awards'].apply(lambda x: ("AS" in x) + 0 if pd.notna(x) else 0)
df_2023['AwardWinner'] = df_2023['Awards'].apply(lambda x: check_award_winner(x) if pd.notna(x) else 0)
df_2023['FirstTeam'] = df_2023['Awards'].apply(lambda x: all_nba_team_1(x) if pd.notna(x) else 0)
df_2023['SecondTeam'] = df_2023['Awards'].apply(lambda x: all_nba_team_2(x) if pd.notna(x) else 0)
df_2023['ThridTeam'] = df_2023['Awards'].apply(lambda x: all_nba_team_3(x) if pd.notna(x) else 0)
df_2023['DefTeam1'] = df_2023['Awards'].apply(lambda x: all_defensive_1(x) if pd.notna(x) else 0)
df_2023['DefTeam2'] = df_2023['Awards'].apply(lambda x: all_defensive_2(x) if pd.notna(x) else 0)

In [29]:
import re

def is_multi_team(team):
    return bool(re.match(r'\d+TM', str(team)))

def fix_team_labels(df):
    cleaned_rows = []
    
    # Group by player and season
    grouped = df.groupby(['Player'])
    
    for (player), group in grouped:
        multi_team_row = group[group['Team'].apply(is_multi_team)]
        if not multi_team_row.empty:
            other_teams = group[~group['Team'].apply(is_multi_team)]
            if not other_teams.empty:
                last_team = other_teams.iloc[-1]['Team']
            else:
                last_team = None
            
            row = multi_team_row.iloc[0].copy()
            row['Team'] = last_team
            cleaned_rows.append(row)
        else:
            cleaned_rows.append(group.iloc[0])
    
    return pd.DataFrame(cleaned_rows)

df_cleaned_2024 = fix_team_labels(df_2024)
df_cleaned_2023 = fix_team_labels(df_2023)

In [30]:
full2024 = pd.merge(df_cleaned_2024, salaries24, on=['Player'])
full2024 = full2024.drop(columns=['Rk', 'Tm', '2024-25', '2025-26', '2026-27', '2027-28', '2028-29'])
full2024 = full2024.rename(columns={'2023-24':"Salary"})
full2024.to_csv("final_2024_player.csv", index=False)

In [35]:
salaries25_updated = salaries25.loc[:, ['Player', '2024-25', 'Guaranteed']]
salaries25_updated = salaries25_updated.rename(columns={'2024-25':"Next_Year_Salary", "Guaranteed":"Next_Year_Guaranteed"})
full2024_updated = pd.merge(full2024, salaries25_updated, on=['Player'], how='inner')
full2024_updated.to_csv("final_2024_player.csv", index=False)

In [34]:
full2023 = pd.merge(df_cleaned_2023, salaries23, on=['Player'])
full2023 = full2023.drop(columns=['Rk', 'Tm', '2023-24', '2024-25', '2025-26', '2026-27', '2027-28'])
full2023 = full2023.rename(columns={'2022-23':"Salary"})
full2023.to_csv("final_2023_player.csv", index=False)
salaries24_updated = salaries24.loc[:, ['Player', '2023-24', 'Guaranteed']]
salaries24_updated = salaries24_updated.rename(columns={'2023-24':"Next_Year_Salary", "Guaranteed":"Next_Year_Guaranteed"})
full2023_updated = pd.merge(full2023, salaries24_updated, on=['Player'], how='inner')
full2023_updated.to_csv("final_2023_player.csv", index=False)
